Dynamische Bin width

In [4]:
import pandas as pd
# dynamische bin width
def process_germanium_rows(germanium_rows):
    if germanium_rows.empty:
        return pd.DataFrame(columns=["T0", "Edep_Sum", "Volumen"])

    # Sortiere nach Volumen und Zeit
    germanium_rows = germanium_rows.sort_values(by=["Volumen", "Time"])

    bins = []  # Liste für die Ergebnisse
    current_bin_energy = 0  # Gesammelte Energie
    t0 = None  # Startzeit des aktuellen Bins
    last_time = None  # Zeit der letzten Energie-Deposition
    current_volume = None  # Aktuelles Volumen

    for _, row in germanium_rows.iterrows():
        time = row["Time"]
        energy = row["Edep"]
        volume = row["Volumen"]

        if t0 is None:
            # Erster Eintrag: Initialisierung
            t0 = time
            current_bin_energy = energy
            last_time = time
            current_volume = volume
        else:
            # Falls wir noch im selben Volumen sind und die Zeitdifferenz < 10 µs bleibt -> addiere Energie
            if volume == current_volume and (time - last_time) < 10_000:
                current_bin_energy += energy
                last_time = time
            else:
                # Zeitdifferenz >= 10 µs oder neues Volumen -> Speichere bisherigen Bin und starte neuen
                bins.append({"T0": t0, "Edep_Sum": current_bin_energy, "Volumen": current_volume})
                
                # Starte neuen Bin
                t0 = time
                current_bin_energy = energy
                last_time = time
                current_volume = volume

    # Speichere den letzten Bin
    if t0 is not None:
        bins.append({"T0": t0, "Edep_Sum": current_bin_energy, "Volumen": current_volume})

    return pd.DataFrame(bins)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

number = ["30", "40", "50", "60", "70", "80", "90", "100", "110", "120"]
number = ["30", "40", "50", "60", "70"]
# number = ["30"]
for i in range(len(number)):
    filename_germanium = f"/home/loris/sim/l200_withoutOpticalProcesses_Grabmayr/build/output{number[i]}_nt_GermaniumHits.csv"


    # Einlesen der Germanium-Datei
    data_germanium = pd.read_csv(
        filename_germanium,
        comment='#',
        header=None,
        names=["Eventnumber", "Time", "generationMarker", "Edep", "Volumen"],
        dtype={
            "Eventnumber": "int64",           # Eventnummern sind typischerweise Ganzzahlen.            
            "Time": "float64",                # Zeit ist ebenfalls eine Fließkommazahl.           
            "generationMarker": "int64",      # Marker sind Ganzzahlen.            
            "Edep": "float64",                 # Deposited Energy ist eine Fließkommazahl.
            "Volumen": "string"
        }
    )    

    all_germanium_rows = []  # Liste für alle gefundenen Rows
    all_germanium_rows_with_marker = []  # Liste für alle gefundenen Rows
    all_germanium_rows_with_zero_marker = []

    maxtime = 150000


    number_of_Edep_Events = data_germanium["Eventnumber"].unique()
    print(len(number_of_Edep_Events))
    for event_number in number_of_Edep_Events:
        
        # Überprüfen, ob die Eventnumber in data_germanium existiert
        germanium_rows = data_germanium[(data_germanium['Eventnumber'] == event_number) &                                    
                                        (data_germanium['Time'] < maxtime) &
                                        (data_germanium['Edep'] > 0)]
        
        if germanium_rows['generationMarker'].dtype != 'int64':
            print(germanium_rows['generationMarker'].dtype)


        germanium_rows_with_marker = germanium_rows[(germanium_rows['generationMarker'] != 0)]
                
        germanium_rows_with_zero_marker = germanium_rows[(germanium_rows['generationMarker'] == 0)]
        
        germanium_rows_summed = process_germanium_rows(germanium_rows)
        germanium_rows_with_marker_summed = process_germanium_rows(germanium_rows_with_marker)
        germanium_rows_with_zero_marker_summed = process_germanium_rows(germanium_rows_with_zero_marker)

        # Füge die Zeilen zur Liste hinzu
        all_germanium_rows.append(germanium_rows_summed)               

        # Füge die Zeilen zur Liste hinzu
        all_germanium_rows_with_marker.append(germanium_rows_with_marker_summed)    

        # Füge die Zeilen zur Liste hinzu
        all_germanium_rows_with_zero_marker.append(germanium_rows_with_zero_marker_summed)             
        
    # Alle DataFrames zusammenführen
    all_germanium_rows_df_Edep_Events = pd.concat(all_germanium_rows, ignore_index=True)
    all_germanium_rows_with_marker_df_Edep_Events = pd.concat(all_germanium_rows_with_marker, ignore_index=True)
    all_germanium_rows_with_zero_marker_df_Edep_Events = pd.concat(all_germanium_rows_with_zero_marker, ignore_index=True)

    # Speichern als CSV-Dateien
    all_germanium_rows_df_Edep_Events.to_csv(f'csv-files/eachSim/germanium_rows_Edep_Events_{number[i]}.csv', index=False)
    all_germanium_rows_with_marker_df_Edep_Events.to_csv(f'csv-files/eachSim/germanium_rows_with_marker_Edep_Events_{number[i]}.csv', index=False)
    all_germanium_rows_with_zero_marker_df_Edep_Events.to_csv(f'csv-files/eachSim/germanium_rows_with_zero_marker_Edep_Events_{number[i]}.csv', index=False)

    print(f"len(all_germanium_rows_df_Edep_Events) = {len(all_germanium_rows_df_Edep_Events)}")
    print(f"len(all_germanium_rows_with_marker_df_Edep_Events) = {len(all_germanium_rows_with_marker_df_Edep_Events)}")
    print(f"len(all_germanium_rows_with_zero_marker_df_Edep_Events) = {len(all_germanium_rows_with_zero_marker_df_Edep_Events)}")

    
    # missing_rows = germanium_rows[germanium_rows['generationMarker'].isna()]
    # print(len(missing_rows))

       

Für alle Edep aus data_germanium

Kombiniere alle Teilergebnisse von oben

In [ ]:
import pandas as pd

# Allgemeine Funktion zum Kombinieren von Dateien
def combine_csv_files(file_pattern, output_filename, numbers):
    # Erstellen der Dateiliste basierend auf dem Muster und den Nummern
    file_list = [file_pattern.format(number=n) for n in numbers]
    
    # Leere Liste, um die DataFrames zu speichern
    dataframes = []
    
    # Dateien einlesen und DataFrames zur Liste hinzufügen
    for file in file_list:
        df = pd.read_csv(file)
        dataframes.append(df)
    
    # Alle DataFrames zusammenfügen
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    # Ergebnis in eine neue CSV-Datei speichern
    combined_df.to_csv(output_filename, index=False)
    print(f"Die Dateien wurden erfolgreich kombiniert und in {output_filename} gespeichert.")

# Liste der Nummern
numbers = [30, 40, 50, 60, 70]

# Dateien kombinieren
combine_csv_files(f"csv-files/eachSim/germanium_rows_Edep_Events_{{number}}_.csv", 
                  f"csv-files/combined/germanium_rows_Edep_Events_all.csv", 
                  numbers)

combine_csv_files(f"csv-files/eachSim/germanium_rows_with_marker_Edep_Events_{{number}}.csv", 
                  f"csv-files/combined/germanium_rows_with_marker_Edep_Events_all.csv", 
                  numbers)

combine_csv_files(f"csv-files/eachSim/germanium_rows_with_zero_marker_Edep_Events_{{number}}.csv", 
                  f"csv-files/combined/germanium_rows_with_zero_marker_Edep_Events_all.csv", 
                  numbers)


Optimale Parameter Finden

In [3]:
import pandas as pd
import numpy as np
from itertools import product
import math

# CSV-Dateien einlesen
all_germanium_rows_df_Edep_Events_all = pd.read_csv(f"csv-files/combined/germanium_rows_Edep_Events_all.csv")
all_germanium_rows_with_marker_df_Edep_Events_all = pd.read_csv(f"csv-files/combined/germanium_rows_with_marker_Edep_Events__all.csv")
all_germanium_rows_with_zero_marker_df_Edep_Events_all = pd.read_csv(f"csv-files/combined/germanium_rows_with_zero_marker_Edep_Events_all.csv")

energy_step_width = 100
time_step_width = 1000
E_max_with_marker = math.ceil(all_germanium_rows_with_marker_df_Edep_Events_all['Edep_Sum'].max()*1000 / energy_step_width) * energy_step_width
E_min_with_marker = math.floor(all_germanium_rows_with_marker_df_Edep_Events_all['Edep_Sum'].min()*1000 / energy_step_width) * energy_step_width
# T_max_with_marker = math.ceil(all_germanium_rows_with_marker_df_Edep_Events_all['T0'].max() / time_step_width) * time_step_width
T_max_with_marker = 60000
T_min_with_marker = math.floor(all_germanium_rows_with_marker_df_Edep_Events_all['T0'].min() / time_step_width) * time_step_width
# Datei im Anhängemodus öffnen (a = append)
with open("optimaleParameter.txt", "a") as file:
    file.write(f"""Ergebnisssse für Gerda Setup:\n
                Es wir ein Energie-Bereich von {E_min_with_marker} bis {E_max_with_marker}\n
                und ein Zeit-Bereich von {T_min_with_marker} bis {T_max_with_marker} betrachtet.\n""")

# Eingabegrenzen (kann angepasst werden, um feiner zu optimieren)
E_min_range = np.arange(E_min_with_marker, E_max_with_marker-energy_step_width, energy_step_width)    # Wertebereich für E_min
E_max_range = np.arange(E_min_with_marker+energy_step_width, E_max_with_marker, energy_step_width)   # Wertebereich für E_max
T_min_range = np.arange(T_min_with_marker, T_max_with_marker-time_step_width, time_step_width) # Wertebereich für T_min
T_max_range = np.arange(T_min_with_marker+time_step_width, T_max_with_marker, time_step_width) # Wertebereich für T_max

# Funktion zur Berechnung des Verhältnisses und der Signalanzahl
def calculate_ratio(E_min, E_max, T_min, T_max):
    # Filterdaten basierend auf den Eingabeparametern
    filtered_all = all_germanium_rows_df_Edep_Events_all[
        (all_germanium_rows_df_Edep_Events_all['Edep_Sum'] * 1000 > E_min) &
        (all_germanium_rows_df_Edep_Events_all['Edep_Sum'] * 1000 < E_max) &
        (all_germanium_rows_df_Edep_Events_all['T0'] > T_min) &
        (all_germanium_rows_df_Edep_Events_all['T0'] < T_max)
    ]

    filtered_marker = all_germanium_rows_with_marker_df_Edep_Events_all[
        (all_germanium_rows_with_marker_df_Edep_Events_all['Edep_Sum'] * 1000 > E_min) &
        (all_germanium_rows_with_marker_df_Edep_Events_all['Edep_Sum'] * 1000 < E_max) &
        (all_germanium_rows_with_marker_df_Edep_Events_all['T0'] > T_min) &
        (all_germanium_rows_with_marker_df_Edep_Events_all['T0'] < T_max)
    ]

    filtered_zero_marker = all_germanium_rows_with_zero_marker_df_Edep_Events_all[
        (all_germanium_rows_with_zero_marker_df_Edep_Events_all['Edep_Sum'] * 1000 > E_min) &
        (all_germanium_rows_with_zero_marker_df_Edep_Events_all['Edep_Sum'] * 1000 < E_max) &
        (all_germanium_rows_with_zero_marker_df_Edep_Events_all['T0'] > T_min) &
        (all_germanium_rows_with_zero_marker_df_Edep_Events_all['T0'] < T_max)
    ]

    N_Edep = len(filtered_all)
    N_Edep_onlyGdNc = len(filtered_marker)
    N_Edep_withoutGdNc = len(filtered_zero_marker)

    # Sicherstellen, dass Division durch 0 vermieden wird
    if N_Edep_withoutGdNc == 0:
        return 0, N_Edep

    ratio = N_Edep_onlyGdNc / N_Edep_withoutGdNc
    return ratio, N_Edep

# Maximierung
best_params = None
max_ratio = -np.inf

for E_min, E_max, T_min, T_max in product(E_min_range, E_max_range, T_min_range, T_max_range):  
    if E_min >= E_max or T_min >= T_max:
        continue  # Überspringe ungültige Kombinationen

    ratio, N_Edep = calculate_ratio(E_min, E_max, T_min, T_max)

    # Überprüfen, ob Bedingung erfüllt ist und ratio maximiert wird
    if N_Edep > 1000 and ratio > max_ratio:
        max_ratio = ratio
        best_params = (E_min, E_max, T_min, T_max)

# Ergebnisse ausgeben
if best_params:
    with open("optimaleParameter.txt", "a") as file:
        file.write(f"""Optimale Parameter:\n
                        E_min: {best_params[0]} keV, E_max: {best_params[1]} keV \n
                        T_min: {best_params[2]} ns, T_max: {best_params[3]} ns \n
                        Maximales Verhältnis (ratio): {max_ratio}\n\n""")
else:
    print("Keine gültigen Parameter gefunden, die die Bedingungen erfüllen.\n")
    


Writing Signal to Noise Csv 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import csv


# E_min = best_params[0] #keV
# E_max = best_params[1]
# T_min = best_params[2] #ns
# T_max = best_params[3]


# Manuelle Eingabe:

E_min = 900 #keV
E_max = 5700
T_min = 500 #ns
T_max = 53000


print(f"E_min: {E_min} keV")
print(f"E_max: {E_max} keV")
print(f"T_min: {T_min} ns")
print(f"T_max: {T_max} ns")

# Datei einlesen
all_germanium_rows_df_Edep_Events_all = pd.read_csv(f"csv-files/combined/germanium_rows_Edep_Events_all.csv")
all_germanium_rows_with_marker_df_Edep_Events_all = pd.read_csv(f"csv-files/combined/germanium_rows_with_marker_Edep_Events_all.csv")
all_germanium_rows_with_zero_marker_df_Edep_Events_all = pd.read_csv(f"csv-files/combined/germanium_rows_with_zero_marker_Edep_Events_all.csv")
# numberOfStrings = "4"
# Datei einlesen
# all_germanium_rows_df_Edep_Events_all = pd.read_csv(f"csv-files/eachSim/germanium_rows_Edep_Events_gerdaSetup_sphericalCylinder_16x16x20_musundynamic.csv")
# all_germanium_rows_with_marker_df_Edep_Events_all = pd.read_csv(f"csv-files/eachSim/germanium_rows_with_marker_Edep_Events_gerdaSetup_sphericalCylinder_16x16x20_musundynamic.csv")
# all_germanium_rows_with_zero_marker_df_Edep_Events_all = pd.read_csv(f"csv-files/eachSim/germanium_rows_with_zero_marker_Edep_Events_gerdaSetup_sphericalCylinder_16x16x20_musundynamic.csv")


print(max(all_germanium_rows_df_Edep_Events_all['Edep_Sum']))

all_germanium_rows_df_Edep_Events_all_filtered = all_germanium_rows_df_Edep_Events_all[
    (all_germanium_rows_df_Edep_Events_all['Edep_Sum'] * 1000 > E_min) &
    (all_germanium_rows_df_Edep_Events_all['Edep_Sum'] * 1000 < E_max) &
    (all_germanium_rows_df_Edep_Events_all['T0'] > T_min) &
    (all_germanium_rows_df_Edep_Events_all['T0'] < T_max)    
]

all_germanium_rows_with_marker_df_Edep_Events_all_filtered = all_germanium_rows_with_marker_df_Edep_Events_all[
    (all_germanium_rows_with_marker_df_Edep_Events_all['Edep_Sum'] * 1000 > E_min) &
    (all_germanium_rows_with_marker_df_Edep_Events_all['Edep_Sum'] * 1000 < E_max) &
    (all_germanium_rows_with_marker_df_Edep_Events_all['T0'] > T_min) &
    (all_germanium_rows_with_marker_df_Edep_Events_all['T0'] < T_max)     
]

all_germanium_rows_with_zero_marker_df_Edep_Events_all_filtered = all_germanium_rows_with_zero_marker_df_Edep_Events_all[
    (all_germanium_rows_with_zero_marker_df_Edep_Events_all['Edep_Sum'] * 1000 > E_min) &
    (all_germanium_rows_with_zero_marker_df_Edep_Events_all['Edep_Sum'] * 1000 < E_max) &
    (all_germanium_rows_with_zero_marker_df_Edep_Events_all['T0'] > T_min) &
    (all_germanium_rows_with_zero_marker_df_Edep_Events_all['T0'] < T_max)    
]

sim_life_time = 22.65 / 2
N_Edep = len(all_germanium_rows_df_Edep_Events_all_filtered)
N_Edep_onlyGdNc = len(all_germanium_rows_with_marker_df_Edep_Events_all_filtered)
N_Edep_withoutGdNc = len(all_germanium_rows_with_zero_marker_df_Edep_Events_all_filtered)
ratio_sim = round(N_Edep_onlyGdNc / N_Edep_withoutGdNc, 4)

print(f'Auf {sim_life_time} Jahre gerechnet:')
print(f'\tNumber of signals: {N_Edep} ')
print(f'\tNumber of signals (only Gd-NC-events): {N_Edep_onlyGdNc} ')
print(f'\tNumber of signals (without Gd-NC-events): {N_Edep_withoutGdNc} ')
print(f'\tRatio of N_Edep_onlyGdNc / N_Edep_withoutGdNC: {ratio_sim} \n')

real_life_time_1 = 0.5
N_Edep_half = round(N_Edep / (sim_life_time / real_life_time_1), 2)
N_Edep_onlyGdNc_half = round(N_Edep_onlyGdNc / (sim_life_time / real_life_time_1), 2)
N_Edep_withoutGdNc_half = round(N_Edep_withoutGdNc / (sim_life_time / real_life_time_1), 2)
ratio_real_1 = round(N_Edep_onlyGdNc_half / N_Edep_withoutGdNc_half, 4)
print(f'Auf {real_life_time_1} Jahre gerechnet:')
print(f'\tNumber of signals: {N_Edep_half} ')
print(f'\tNumber of signals (only Gd-NC-events): {N_Edep_onlyGdNc_half} ')
print(f'\tNumber of signals (without Gd-NC-events): {N_Edep_withoutGdNc_half} ')
print(f'\tRatio of N_Edep_onlyGdNc / N_Edep_withoutGdNc: {ratio_real_1} ')

real_life_time_2 = 0.25
N_Edep_quarter = round(N_Edep / (sim_life_time / real_life_time_2), 2)
N_Edep_onlyGdNc_quarter = round(N_Edep_onlyGdNc / (sim_life_time / real_life_time_2), 2)
N_Edep_withoutGdNc_quarter = round(N_Edep_withoutGdNc / (sim_life_time / real_life_time_2), 2)
ratio_real_2 = round(N_Edep_onlyGdNc_quarter / N_Edep_withoutGdNc_quarter, 4)
print(f'Auf {real_life_time_2} Jahre gerechnet:')
print(f'\tNumber of signals: {N_Edep_quarter} ')
print(f'\tNumber of signals (only Gd-NC-events): {N_Edep_onlyGdNc_quarter} ')
print(f'\tNumber of signals (without Gd-NC-events): {N_Edep_withoutGdNc_quarter} ')
print(f'\tRatio of N_Edep_onlyGdNc / N_Edep_withoutGdNc: {ratio_real_2} ')

real_life_time_3 = 0.056082192 # 20.47 days
N_Edep_3 = round(N_Edep / (sim_life_time / real_life_time_3), 2)
N_Edep_onlyGdNc_3 = round(N_Edep_onlyGdNc / (sim_life_time / real_life_time_3), 2)
N_Edep_withoutGdNc_3 = round(N_Edep_withoutGdNc / (sim_life_time / real_life_time_3), 2)
ratio_real_3 = round(N_Edep_onlyGdNc_3 / N_Edep_withoutGdNc_3, 4)
print(f'Auf {real_life_time_3} Jahre gerechnet:')
print(f'\tNumber of signals: {N_Edep_3} ')
print(f'\tNumber of signals (only Gd-NC-events): {N_Edep_onlyGdNc_3} ')
print(f'\tNumber of signals (without Gd-NC-events): {N_Edep_withoutGdNc_3} ')
print(f'\tRatio of N_Edep_onlyGdNc / N_Edep_withoutGdNc: {ratio_real_3} ')

data = [
    # ["Number of strings", 
    #  "E_min [keV]",
    #  "E_max [keV]", 
    #  "T_min [ns]", 
    #  "T_max [ns]",
    #  "lifetime [yr]",
    #  "Number of signals", 
    #  "Number of signals (only Gd-NC-events)",
    #  "Number of signals (without Gd-NC-events)", 
    #  "Ratio of N_Edep_onlyGdNc / N_Edep_withoutGdNC"],  # Header
    [numberOfStrings, E_min, E_max, T_min, T_max,  sim_life_time, N_Edep, N_Edep_onlyGdNc, N_Edep_withoutGdNc, ratio_sim],
    [numberOfStrings, E_min, E_max, T_min, T_max, real_life_time_1, N_Edep_half, N_Edep_onlyGdNc_half, N_Edep_withoutGdNc_half, ratio_real_1],
    [numberOfStrings, E_min, E_max, T_min, T_max, real_life_time_2, N_Edep_quarter, N_Edep_onlyGdNc_quarter, N_Edep_withoutGdNc_quarter, ratio_real_2 ],
    [numberOfStrings, E_min, E_max, T_min, T_max, real_life_time_3, N_Edep_3, N_Edep_onlyGdNc_3, N_Edep_withoutGdNc_3, ratio_real_3]
]

# In eine CSV-Datei schreiben
with open("signal_to_noise.csv", "a", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)

print("CSV-Datei wurde erstellt.")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

# /home/loris/sim/analysis/csv-files/combined/germanium_rows_Edep_Events_3_Ga_Strings_new_all.csv
all_germanium_rows_df_Edep_Events_all = pd.read_csv(f"csv-files/combined/germanium_rows_Edep_Events_all.csv")
all_germanium_rows_with_marker_df_Edep_Events_all = pd.read_csv(f"csv-files/combined/germanium_rows_with_marker_Edep_Events_all.csv")
all_germanium_rows_with_zero_marker_df_Edep_Events_all = pd.read_csv(f"csv-files/combined/germanium_rows_with_zero_marker_Edep_Events_all.csv")

print(f"len(all_germanium_rows_df_Edep_Events_all) = {len(all_germanium_rows_df_Edep_Events_all)}")
print(f"len(all_germanium_rows_with_marker_df_Edep_Events_all) = {len(all_germanium_rows_with_marker_df_Edep_Events_all)}")
print(f"len(all_germanium_rows_with_zero_marker_df_Edep_Events_all) = {len(all_germanium_rows_with_zero_marker_df_Edep_Events_all)}")
# Logarithmische Y-Bins erstellen
y_min = all_germanium_rows_df_Edep_Events_all["Edep_Sum"].min() * 1000  # kleinster Wert in keV
y_max = all_germanium_rows_df_Edep_Events_all["Edep_Sum"].max() * 1000  # größter Wert in keV
log_y_bins = np.logspace(np.log10(y_min), np.log10(y_max), 50)  # 50 Bins logarithmisch verteilt

# 1. 2D-Histogramm für alle Daten
x_bin_width = 1000
number_x_bins = round(all_germanium_rows_df_Edep_Events_all["T0"].max()/x_bin_width) 
plt.figure(figsize=(7,4))
plt.hist2d(all_germanium_rows_df_Edep_Events_all["T0"], all_germanium_rows_df_Edep_Events_all["Edep_Sum"]*1000, bins=(number_x_bins, log_y_bins), cmap='plasma', norm=mcolors.LogNorm())
plt.colorbar(label="Counts (log scale)")
plt.title("Time vs Deposited Energy (All Data) ($10^8$ muons)")
plt.suptitle(f"all Edep-Ge-Events (ratio: {ratio_sim})")
plt.xlabel("Time (ns)")
plt.ylabel(f"Deposited Energy [keV / {x_bin_width} ns]")
if E_max > 10000:
    plt.plot([T_min, T_min], [E_min, plt.ylim()[1]], color='green', linestyle='-', linewidth=5, label=f"{T_min} ns < T < {T_max} ns")
    plt.plot([T_max, T_max], [E_min, plt.ylim()[1]], color='green', linestyle='-')
    plt.plot([T_min, T_max], [E_min, E_min], color='green', linestyle='-', label=f"E > {E_min} keV")
else: 
    plt.plot([T_min, T_min], [E_min, E_max], color='green', linestyle='-', linewidth=2, label=f"{T_min} ns < T < {T_max} ns")
    plt.plot([T_max, T_max], [E_min, E_max], color='green', linestyle='-', linewidth=2)
    plt.plot([T_min, T_max], [E_max, E_max], color='green', linestyle='-', linewidth=2)
    plt.plot([T_min, T_max], [E_min, E_min], color='green', linestyle='-', linewidth=2, label=f"{E_min} keV < E < {E_max} keV")
# plt.xscale('log')
plt.yscale('log')
plt.xlim(0, 155000)
plt.legend(loc='upper right')
# plt.ylim(0, 1000)


output_plot_path = f"plots/2D/EdepEvents/Time_vs_Edep_allData_Edep_Events_50mio.pdf"
plt.savefig(output_plot_path, format='pdf')
plt.close()

# 2. Daten filtern: generationMarker ungleich null
number_x_bins = round(all_germanium_rows_with_marker_df_Edep_Events_all["T0"].max()/x_bin_width) 
plt.figure(figsize=(7, 4))
plt.hist2d(all_germanium_rows_with_marker_df_Edep_Events_all["T0"], all_germanium_rows_with_marker_df_Edep_Events_all["Edep_Sum"]*1000, bins=(number_x_bins, log_y_bins), cmap='plasma', norm=mcolors.LogNorm())
plt.colorbar(label="Counts")
plt.title("Time vs Deposited Energy of particles caused by Gd-NC ($10^8$ muons)")
plt.suptitle(f"all Edep-Ge-Events (ratio: {ratio_sim})")
plt.xlabel("Time (ns)")
plt.ylabel(f"Deposited Energy [keV / {x_bin_width} ns]")
if E_max > 10000:
    plt.plot([T_min, T_min], [E_min, plt.ylim()[1]], color='green', linestyle='-', label=f"{T_min} ns < T < {T_max} ns")
    plt.plot([T_max, T_max], [E_min, plt.ylim()[1]], color='green', linestyle='-')
    plt.plot([T_min, T_max], [E_min, E_min], color='green', linestyle='-', label=f"E > {E_min} keV")
else: 
    plt.plot([T_min, T_min], [E_min, E_max], color='green', linestyle='-', linewidth=2, label=f"{T_min} ns < T < {T_max} ns")
    plt.plot([T_max, T_max], [E_min, E_max], color='green', linestyle='-', linewidth=2)
    plt.plot([T_min, T_max], [E_max, E_max], color='green', linestyle='-', linewidth=2)
    plt.plot([T_min, T_max], [E_min, E_min], color='green', linestyle='-', linewidth=2, label=f"{E_min} keV < E < {E_max} keV")
plt.yscale('log')
plt.xlim(0, 155000)
plt.legend(loc='upper right')
output_plot_path = f"plots/2D/EdepEvents/Time_vs_Edep_onlyGdNC_Edep_Events_50mio.pdf"
plt.savefig(output_plot_path, format='pdf')
plt.close()

# 3. Daten filtern: generationMarker gleich null
number_x_bins = round(all_germanium_rows_with_zero_marker_df_Edep_Events_all["T0"].max()/x_bin_width) 
plt.figure(figsize=(7, 4))
plt.hist2d(all_germanium_rows_with_zero_marker_df_Edep_Events_all["T0"], all_germanium_rows_with_zero_marker_df_Edep_Events_all["Edep_Sum"]*1000, bins=(number_x_bins, log_y_bins), cmap='plasma', norm=mcolors.LogNorm())
plt.colorbar(label="Counts")
plt.title("Time vs Deposited Energy WITHOUT particles caused by Gd-NC ($10^8$ muons)")
plt.suptitle(f"all Edep-Ge-Events (ratio: {ratio_sim})")
plt.xlabel("Time (ns)")
plt.ylabel(f"Deposited Energy [keV / {x_bin_width} ns]")
if E_max > 10000:
    plt.plot([T_min, T_min], [E_min, plt.ylim()[1]], color='green', linestyle='-', label=f"{T_min} ns < T < {T_max} ns")
    plt.plot([T_max, T_max], [E_min, plt.ylim()[1]], color='green', linestyle='-')
    plt.plot([T_min, T_max], [E_min, E_min], color='green', linestyle='-', label=f"E > {E_min} keV")
else: 
    plt.plot([T_min, T_min], [E_min, E_max], color='green', linestyle='-', linewidth=2, label=f"{T_min} ns < T < {T_max} ns")
    plt.plot([T_max, T_max], [E_min, E_max], color='green', linestyle='-', linewidth=2)
    plt.plot([T_min, T_max], [E_max, E_max], color='green', linestyle='-', linewidth=2)
    plt.plot([T_min, T_max], [E_min, E_min], color='green', linestyle='-', linewidth=2, label=f"{E_min} keV < E < {E_max} keV")
plt.yscale('log')
plt.xlim(0, 155000)
plt.legend(loc='upper right')
output_plot_path = f"plots/2D/EdepEvents/Time_vs_Edep_withoutGdNC_Edep_Events_50mio.pdf"
plt.savefig(output_plot_path, format='pdf')
plt.close()